In [16]:
import torch
import torch.utils.data
from torch.utils.data import TensorDataset

import argparse
import time
import gc
import random

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn

from net import GNNStack
from utils import AverageMeter, accuracy, log_msg, get_default_train_val_test_loader

from tqdm import tqdm

In [2]:
# dsid = "Mortality"

In [3]:
# # get dataset from .pt
# data_train = torch.load(f'data/UCR/{dsid}/X_train.pt')
# data_val = torch.load(f'data/UCR/{dsid}/X_valid.pt')
# label_train = torch.load(f'data/UCR/{dsid}/y_train.pt')
# label_val = torch.load(f'data/UCR/{dsid}/y_valid.pt')

# # init [num_variables, seq_length, num_classes]
# num_nodes = data_val.size(-2)

# seq_length = data_val.size(-1)

# num_classes = len(torch.bincount(label_val.type(torch.int)))

In [4]:
# data_train.shape, data_val.shape, label_train.shape, label_val.shape

In [5]:
# num_nodes ### each feature is a node

In [6]:
# seq_length ### sequence length: number of total graphs

In [7]:
# num_classes ### ri

## aruguments

In [8]:
args = {
    'arch': 'dyGIN2d', #what other models I can put here?? dyGCN2d
    'dataset': 'Mortality',
    'num_layers': 3,  # the number of GNN layers
    'groups': 4,  # the number of time series groups (num_graphs)
    'pool_ratio': 0.2,  # the ratio of pooling for nodes
    'kern_size': [9,5,3],  # list of time conv kernel size for each layer
    'in_dim': 64,  # input dimensions of GNN stacks
    'hidden_dim': 128,  # hidden dimensions of GNN stacks
    'out_dim': 256,  # output dimensions of GNN stacks
    'workers': 0,  # number of data loading workers
    'epochs': 2000,  # number of total epochs to run
    'batch_size': 16,  # mini-batch size, this is the total batch size of all GPUs
    'val_batch_size': 16,  # validation batch size
    'lr': 1e-4,  # initial learning rate
    'weight_decay': 1e-4,  # weight decay
    'evaluate': False,  # evaluate model on validation set
    'seed': 42,  # seed for initializing training
    'gpu': 0,  # GPU id to use
    'use_benchmark': True,  # use benchmark
    'tag': 'date'  # the tag for identifying the log and model files
}

In [9]:
# train_dataset = TensorDataset(data_train, label_train)
# val_dataset   = TensorDataset(data_val, label_val)

In [10]:
# train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=args['batch_size'],shuffle=True, num_workers=args['workers'], pin_memory=True)
# val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=args['val_batch_size'], shuffle=False,num_workers=args['workers'],pin_memory=True)

In [11]:
# def main():
#     # args = parser.parse_args()
    
#     # args.kern_size = [ int(l) for l in args.kern_size.split(",") ]

#     # if args.seed is not None:
#     random.seed(args['seed'])
#     torch.manual_seed(args['seed'])

#     main_work(args)

In [14]:
def main_work(args):
    
    random.seed(args['seed'])
    torch.manual_seed(args['seed'])
    
    
    # init acc
    best_acc1 = 0
    
    if args['tag'] == 'date':
        local_date = time.strftime('%m.%d', time.localtime(time.time()))
        args['tag'] = local_date

    log_file = 'log/{}_gpu{}_{}_{}_exp.txt'.format(args['tag'], args['gpu'], args['arch'], args['dataset'])

    if args['gpu'] is not None:
        print("Use GPU: {} for training".format(args['gpu']))


    # dataset
    train_loader, val_loader, num_nodes, seq_length, num_classes = get_default_train_val_test_loader(args)
    
    print(num_nodes,seq_length,num_classes)
    
    # training model from net.py
    model = GNNStack(gnn_model_type=args['arch'], num_layers=args['num_layers'], 
                     groups=args['groups'], pool_ratio=args['pool_ratio'], kern_size=args['kern_size'], 
                     in_dim=args['in_dim'], hidden_dim=args['hidden_dim'], out_dim=args['out_dim'], 
                     seq_len=seq_length, num_nodes=num_nodes, num_classes=num_classes)

    # print & log
    log_msg('epochs {}, lr {}, weight_decay {}'.format(args['epochs'], args['lr'], args['weight_decay']), log_file)


    # determine whether GPU or not
    if not torch.cuda.is_available():
        print("Warning! Using CPU!!!")
    elif args.gpu is not None:
        torch.cuda.set_device(args['gpu'])

        # collect cache
        gc.collect()
        torch.cuda.empty_cache()

        model = model.cuda(args['gpu'])
        if args['use_benchmark']:
            cudnn.benchmark = True
        print('Using cudnn.benchmark.')
    else:
        print("Error! We only have one gpu!!!")


    # define loss function(criterion) and optimizer
    criterion = nn.CrossEntropyLoss().cuda(args['gpu'])
    
    optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'], weight_decay=args['weight_decay'])
    
    lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=50, verbose=True)

    # validation
    if args['evaluate']:
        validate(val_loader, model, criterion, args)
        return

    # train & valid
    print('****************************************************')
    print('Dataset: ', args['dataset'])

    dataset_time = AverageMeter('Time', ':6.3f')

    loss_train = []
    acc_train = []
    loss_val = []
    acc_val = []
    epoches = []

    end = time.time()
    for epoch in tqdm.notebook(range(args['epochs'])):
        epoches += [epoch]

        # train for one epoch
        acc_train_per, loss_train_per = train(train_loader, model, criterion, optimizer, lr_scheduler, args)
        
        acc_train += [acc_train_per]
        loss_train += [loss_train_per]

        msg = f'TRAIN, epoch {epoch}, loss {loss_train_per}, acc {acc_train_per}'
        log_msg(msg, log_file)


        # evaluate on validation set
        acc_val_per, loss_val_per = validate(val_loader, model, criterion, args)

        acc_val += [acc_val_per]
        loss_val += [loss_val_per]

        msg = f'VAL, loss {loss_val_per}, acc {acc_val_per}'
        log_msg(msg, log_file)

        # remember best acc
        best_acc1 = max(acc_val_per, best_acc1)


    # measure elapsed time
    dataset_time.update(time.time() - end)

    # log & print the best_acc
    msg = f'\n\n * BEST_ACC: {best_acc1}\n * TIME: {dataset_time}\n'
    log_msg(msg, log_file)

    print(f' * best_acc1: {best_acc1}')
    print(f' * time: {dataset_time}')
    print('****************************************************')


    # collect cache
    gc.collect()
    torch.cuda.empty_cache()


def train(train_loader, model, criterion, optimizer, lr_scheduler, args):
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc', ':6.2f')

    # switch to train mode
    model.train()

    for count, (data, label) in enumerate(train_loader):

        # data in cuda
        data = data.cuda(args.gpu).type(torch.float)
        label = label.cuda(args.gpu).type(torch.long)

        # compute output
        output = model(data)
    
        loss = criterion(output, label)

        # measure accuracy and record loss
        acc1 = accuracy(output, label, topk=(1, 1))
        losses.update(loss.item(), data.size(0))
        top1.update(acc1[0], data.size(0))

        # compute gradient and do Adam step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    lr_scheduler.step(top1.avg)

    return top1.avg, losses.avg


def validate(val_loader, model, criterion, args):
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        for count, (data, label) in enumerate(val_loader):
            if args.gpu is not None:
                data = data.cuda(args.gpu, non_blocking=True).type(torch.float)
            if torch.cuda.is_available():
                label = label.cuda(args.gpu, non_blocking=True).type(torch.long)

            # compute output
            output = model(data)

            loss = criterion(output, label)

            # measure accuracy and record loss
            acc1 = accuracy(output, label, topk=(1, 1))
            losses.update(loss.item(), data.size(0))
            top1.update(acc1[0], data.size(0))

    return top1.avg, losses.avg

In [15]:
main_work(args)

Use GPU: 0 for training
231 288 2


AttributeError: 'dict' object has no attribute 'epochs'